### Подключение библиотек

In [1]:
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras import regularizers
from keras.layers import Conv1D, Flatten, Dense
from keras.models import Sequential, load_model

In [9]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception:
    pass

In [2]:
NUM_CLASSES = 4

In [3]:
# path = 'drive/Othercomputers/Ноутбук/Artist/'
path = '.'

### Обработка данных для статичного классификатора

In [9]:
dsopen = np.load(path + '/dataset/open.npy')
dsok = np.load(path + '/dataset/ok.npy')
dsup = np.load(path + '/dataset/up.npy')
dsdis = np.load(path + '/dataset/dislike.npy')

In [10]:
dsall = np.vstack((dsopen, dsup, dsok, dsdis))
dsall[:, :, 0] /= 1920
dsall[:, :, 1] /= 1080
dsall.shape

(1873, 21, 3)

In [12]:
dsallx = dsall.copy()
dsally = np.array([[1, 0, 0, 0]] * dsopen.shape[0] + [[0, 1, 0, 0]] * dsup.shape[0] + [[0, 0, 1, 0]] * dsok.shape[0] + [[0, 0, 0, 1]] * dsdis.shape[0], dtype='float32')

In [13]:
dsallx = dsallx.astype('float32')
dsally = dsally.astype('float32')

In [15]:
np.save(path + '/dataset/static_X.npy', dsallx)
np.save(path + '/dataset/static_Y.npy', dsally)

### Обучение модели

In [16]:
model_path = path + '/mlmodels/static.keras'

In [17]:
x_train, x_test, y_train, y_test = train_test_split(dsallx, dsally, test_size = 0.3)

In [18]:
model = Sequential([
    Conv1D(60, 2, activation='relu', input_shape = x_train.shape[1:], kernel_regularizer = regularizers.L2(4e-4), dtype='float32'),
    Conv1D(50, 2, activation='relu', kernel_regularizer = regularizers.L2(4e-4)),
    # MaxPooling1D(2),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(
    x_train, y_train,
    epochs = 20,
    validation_data=(x_test, y_test)
)

Epoch 1/20


/opt/homebrew/anaconda3/envs/artist/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3812 - loss: 1.3384 - val_accuracy: 0.8523 - val_loss: 0.8967
Epoch 2/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8983 - loss: 0.6440 - val_accuracy: 0.9911 - val_loss: 0.1121
Epoch 3/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9914 - loss: 0.1039 - val_accuracy: 0.9964 - val_loss: 0.0470
Epoch 4/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9955 - loss: 0.0518 - val_accuracy: 0.9858 - val_loss: 0.0687
Epoch 5/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9933 - loss: 0.0575 - val_accuracy: 0.9875 - val_loss: 0.0502
Epoch 6/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9900 - loss: 0.0579 - val_accuracy: 0.9964 - val_loss: 0.0381
Epoch 7/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9949 - loss: 0.0502 - val_accuracy: 0.9964 - val_loss: 0.0370
Epoch 8/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9884 - loss: 0.0610 - val_accuracy: 0.9982 - val_loss: 0.0363
Epo

In [40]:
model.predict(dsup[[0], :, :2])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


array([[1.4511340e-04, 9.9985266e-01, 2.2599877e-06, 3.8629619e-15]],
      dtype=float32)

In [19]:
model.save(model_path)

### Stable Diffusion

In [2]:
%pip install diffusers transformers accelerate

In [1]:
from PIL import Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch
from skimage.filters import threshold_otsu
import matplotlib.pyplot as plt
import numpy as np

image = Image.open('scribble.png')
image = np.array(image)
c = image[:, :, 0]
t = threshold_otsu(c)
image = Image.fromarray((image <= t).astype('uint8') * 255)

2024-05-31 17:03:35.905951: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-scribble", torch_dtype=torch.float32
).to('cuda')

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, safety_checker=None, torch_dtype=torch.float32
).to('cuda')

pipe.enable_model_cpu_offload()
pipe.enable_xformers_memory_efficient_attention()

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

/home/olegg/.local/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


AttributeError: 'StableDiffusionControlNetPipeline' object has no attribute 'enable_vae_tiling'

In [4]:
imgs = pipe("flower, sketch art", image, num_inference_steps=20, height=320, width=320).images
imgs[0]

  0%|          | 0/20 [00:00<?, ?it/s]

RuntimeError: 

torch._dynamo.optimize is called on a non function object.
If this is a callable class, please wrap the relevant code into a function and optimize the
wrapper function.

>> class CallableClass:
>>     def __init__(self):
>>         super().__init__()
>>         self.relu = torch.nn.ReLU()
>>
>>     def __call__(self, x):
>>         return self.relu(torch.sin(x))
>>
>>     def print_hello(self):
>>         print("Hello world")
>>
>> mod = CallableClass()

If you want to optimize the __call__ function and other code, wrap that up in a function

>> def wrapper_fn(x):
>>     y = mod(x)
>>     return y.sum()

and then optimize the wrapper_fn

>> opt_wrapper_fn = torch._dynamo.optimize(wrapper_fn)
